In [ ]:
# --- Cài thư viện ---
!pip install ctgan --quiet
!pip install pandas --quiet

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import random
from ctgan import CTGAN
import os

# --- Cấu hình ---
input_csv = "/content/drive/MyDrive/consolidated_traffic_data.csv"
output_dir = "/content/drive/MyDrive/synthetic_per_label_1M5"
min_total = 108000
max_total = 113000
random_seed = 42

os.makedirs(output_dir, exist_ok=True)
random.seed(random_seed)

# --- Đọc dataset ---
df = pd.read_csv(input_csv)

print("Các cột có sẵn trong DataFrame:")
print(df.columns)

# --- Thống kê số dòng mỗi nhãn ---
label_counts = df['traffic_type'].value_counts()
print("Số dòng hiện tại mỗi nhãn:")
print(label_counts)

# Danh sách các cột nên là số không âm
non_negative_columns = [
    'duration', 'total_fiat', 'total_biat', 'min_fiat', 'min_biat',
    'max_fiat', 'max_biat', 'mean_fiat', 'mean_biat', 'flowPktsPerSecond',
    'flowBytesPerSecond', 'min_flowiat', 'max_flowiat', 'mean_flowiat',
    'std_flowiat', 'min_active', 'mean_active', 'max_active', 'std_active',
    'min_idle', 'mean_idle', 'max_idle', 'std_idle'
]

# --- Sinh dữ liệu mới cho mỗi nhãn ---
for label, count in label_counts.items():
    target_total = random.randint(min_total, max_total)
    rows_to_generate = max(target_total - count, 0)

    print(f"\nNhãn '{label}': hiện có {count} dòng, muốn tổng ~{target_total} dòng, cần sinh thêm {rows_to_generate} dòng.")

    df_label = df[df['traffic_type'] == label]

    if rows_to_generate > 0:
        # Fit CTGAN trên dữ liệu nhãn này, chỉ định 'traffic_type' là cột rời rạc
        model = CTGAN(epochs=300, batch_size=500)
        model.fit(df_label, discrete_columns=['traffic_type'])

        # Sinh dữ liệu mới
        synthetic_label_data = model.sample(rows_to_generate)

        # Xử lý hậu kỳ: Đảm bảo các cột không âm
        for col in non_negative_columns:
            if col in synthetic_label_data.columns:
                synthetic_label_data[col] = synthetic_label_data[col].apply(lambda x: max(x, 0))
    else:
        synthetic_label_data = pd.DataFrame(columns=df.columns)

    # Gộp dữ liệu gốc + dữ liệu mới
    df_total_label = pd.concat([df_label, synthetic_label_data], ignore_index=True)

    # Xuất CSV riêng
    output_csv = os.path.join(output_dir, f"{label}_synthetic.csv")
    df_total_label.to_csv(output_csv, index=False)

    print(f"Xuất xong {len(df_total_label)} dòng vào {output_csv}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
Mounted at /content/drive
Các cột có sẵn trong DataFrame:
Index(['duration', 'total_fiat', 'total_biat', 'min_fiat', 'min_biat',
       'max_fiat', 'max_biat', 'mean_fiat', 'mean_biat', 'flowPktsPerSecond',
       'flowBytesPerSecond', 'min_flowiat', 'max_flowiat', 'mean_flowiat',
       'std_flowiat', 'min_active', 'mean_active', 'max_active', 'std_active',
       'min_idle', 'mean_idle', 'max_idle', 'std_idle', 'traffic_type'],
      dtype='object')
Số dòng hiện tại mỗi nhãn:
traffic_type
BROWSING         10000
VPN-BROWSING      9982
VOIP              6436
VPN-VOIP          5549
VPN-FT            4616
P2P               3987
FT                3348
VPN-CHAT          2708
VPN-P2P           2486
CHAT              2469
VPN-MAIL          2432
MAIL              1307
STREAMING         1163
VPN-STREAMING     1113
Name: count, dtype: int64

